In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\korean_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'korean']

In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)  
data

,emd_cd,korean
base_date,,
2018-01-31,50110250,7.209306e+06
2018-02-28,50110250,5.579125e+06
2018-03-31,50110250,7.615021e+06
2018-04-30,50110250,8.498349e+06
2018-05-31,50110250,8.916782e+06
...,...,...
2021-02-28,50130620,4.321265e+06
2021-03-31,50130620,4.744309e+06
2021-04-30,50130620,5.236706e+06


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50130540  ##각행정동코드

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,korean
base_date,
2018-01-31,1.711666e+06
2018-02-28,1.123535e+06
2018-03-31,1.169120e+06
2018-04-30,1.357372e+06
2018-05-31,1.453481e+06
2018-06-30,1.311343e+06
2018-07-31,1.324548e+06
2018-08-31,1.510573e+06
2018-09-30,1.413112e+06


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'korean': base_date
 2018-01-31    1.711666e+06
 2018-02-28    1.123535e+06
 2018-03-31    1.169120e+06
 2018-04-30    1.357372e+06
 2018-05-31    1.453481e+06
 2018-06-30    1.311343e+06
 2018-07-31    1.324548e+06
 2018-08-31    1.510573e+06
 2018-09-30    1.413112e+06
 2018-10-31    1.463221e+06
 2018-11-30    1.367715e+06
 2018-12-31    1.400755e+06
 2019-01-31    1.636072e+06
 2019-02-28    1.294563e+06
 2019-03-31    1.350338e+06
 2019-04-30    1.310842e+06
 2019-05-31    1.433128e+06
 2019-06-30    1.302779e+06
 2019-07-31    1.450952e+06
 2019-08-31    1.430191e+06
 2019-09-30    1.205185e+06
 2019-10-31    1.386330e+06
 2019-11-30    1.240927e+06
 2019-12-31    1.314883e+06
 2020-01-31    1.450842e+06
 2020-02-29    1.141289e+06
 2020-03-31    1.105821e+06
 2020-04-30    1.003253e+06
 2020-05-31    1.165710e+06
 2020-06-30    1.148969e+06
 2020-07-31    1.336788e+06
 2020-08-31    1.571017e+06
 2020-09-30    1.237954e+06
 2020-10-31    1.433996e+06
 2020-11-30    1.440281e+06

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -5.435309
p-value : 0.000003
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [8]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=1122.182, Time=0.02 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=1122.617, Time=0.10 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=1124.745, Time=0.04 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=1124.321, Time=0.05 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=1127.340, Time=0.07 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=1129.115, Time=0.13 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=1122.527, Time=0.03 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=1125.566, Time=0.06 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=1127.186, Time=0.10 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=1127.154, Time=0.13 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=1129.412, Time=0.20 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=1126.107, Time=0.06 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=1127.566, Time=0.13 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=1127.728, Time=0.18 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=1128.777, Time=0.24 sec
 ARIMA(3,0

ARIMA(order=(0, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [9]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.korean.values, order=(0,0,0))   

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(0, 0)   Log Likelihood                -559.091
Method:                           css   S.D. of innovations         146203.115
Date:                Sat, 11 Sep 2021   AIC                           1122.182
Time:                        02:33:42   BIC                           1125.657
Sample:                             0   HQIC                          1123.456
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.352e+06   2.26e+04     59.920      0.000    1.31e+06     1.4e+06


C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [10]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([1351781.28152857, 1351781.28152857]), array([146203.11495683, 146203.11495683]), array([[1065228.4417856 , 1638334.12127153],
       [1065228.4417856 , 1638334.12127153]]))
